In [1]:
import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.models import load_model
import yfinance as yf
import os

In [2]:
# Load Data
def load_data(ticker):
    df = yf.download(ticker, start="2010-01-01", end="2022-03-24")
    df = df[['Close']]
    return df

In [3]:
# Preprocess Data
def preprocess_data(df):
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(df)
    return scaled_data, scaler

In [4]:
# Create Dataset for LSTM
def create_dataset(data, time_step=100):
    X, y = [], []
    for i in range(len(data) - time_step - 1):
        X.append(data[i:(i + time_step), 0])
        y.append(data[i + time_step, 0])
    return np.array(X), np.array(y)

In [5]:
# Build LSTM Model
def build_model():
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True, input_shape=(100, 1)))
    model.add(LSTM(units=50, return_sequences=False))
    model.add(Dense(units=25))
    model.add(Dense(units=1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [6]:
# Train Model
def train_model(model, X_train, y_train, epochs=10, batch_size=64):
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size)
    return model

In [7]:
# Save Model
def save_model(model, filename='stock_lstm_model.h5'):
    model.save(filename)

In [8]:
# Predict
def predict_stock(model, data, time_step):
    predictions = []
    for i in range(time_step, len(data)):
        input_data = data[i - time_step:i, 0].reshape(1, time_step, 1)
        predictions.append(model.predict(input_data))
    return np.array(predictions).reshape(-1)

In [9]:
# Sidebar for input
ticker = st.sidebar.text_input("Enter Stock Ticker", "MSFT")
model_file = st.sidebar.file_uploader("Upload pre-trained model (.h5)", type=["h5"])


2025-02-26 13:41:04.238 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-26 13:41:04.239 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-26 13:41:04.240 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-26 13:41:04.241 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-26 13:41:04.241 WARNING streamlit.runtime.state.session_state_proxy: Session state does not function when running a script without `streamlit run`
2025-02-26 13:41:04.243 WARNING streamlit.runtime.scriptrunner_utils.script_run_c

In [10]:
 # Load and display data
st.write(f"Stock Price Data for {ticker}")
df = load_data(ticker)
st.line_chart(df['Close'])


2025-02-26 13:41:10.253 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-26 13:41:10.254 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-26 13:41:10.255 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-26 13:41:10.257 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed
2025-02-26 13:41:11.329 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-26 13:41:11.335 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DeltaGenerator()

In [11]:
# Preprocess data
data, scaler = preprocess_data(df)
time_step = 100
X, y = create_dataset(data, time_step)

In [12]:
# Reshape data
X = np.reshape(X, (X.shape[0], X.shape[1], 1))

In [13]:
# Load or train the model
if model_file:
        model = load_model(model_file)
        st.success("Model Loaded!")
else:
        model = build_model()
        model = train_model(model, X, y)
        save_model(model)
        st.success("Model Trained and Saved!")

Epoch 1/10


c:\Stock prediction\venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


47/47 ━━━━━━━━━━━━━━━━━━━━ 6s 55ms/step - loss: 0.0244
Epoch 2/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 2s 49ms/step - loss: 2.4081e-04
Epoch 3/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 2s 50ms/step - loss: 1.5748e-04
Epoch 4/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 2s 53ms/step - loss: 1.7040e-04
Epoch 5/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 2s 51ms/step - loss: 1.6667e-04
Epoch 6/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 2s 50ms/step - loss: 1.6663e-04
Epoch 7/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 2s 53ms/step - loss: 1.5333e-04
Epoch 8/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 2s 52ms/step - loss: 2.0306e-04
Epoch 9/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 3s 54ms/step - loss: 1.5222e-04
Epoch 10/10
47/47 ━━━━━━━━━━━━━━━━━━━━ 2s 49ms/step - loss: 1.7344e-04


2025-02-26 13:41:45.234 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-26 13:41:45.234 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [14]:
# Predict future prices
predictions = predict_stock(model, data, time_step)
st.write("Predicted Stock Prices")
predicted_prices = scaler.inverse_transform(predictions.reshape(-1, 1))
st.line_chart(predicted_prices)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 248ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━

2025-02-26 13:51:57.621 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-26 13:51:57.622 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-26 13:51:57.622 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-26 13:51:57.626 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-26 13:51:57.686 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-26 13:51:57.687 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DeltaGenerator()